In [1]:
import findspark
#findspark.init()
findspark.init("/home/sarvani/spark-3.3.1-bin-hadoop3")
import pyspark
sc=pyspark.SparkContext()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Project').getOrCreate()

23/02/12 19:14:35 WARN Utils: Your hostname, sarvani-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/02/12 19:14:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/12 19:14:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/12 19:14:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.json('/home/sarvani/iot_devices.json')

In [3]:
df.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [4]:
print(df)

DataFrame[battery_level: bigint, c02_level: bigint, cca2: string, cca3: string, cn: string, device_id: bigint, device_name: string, humidity: bigint, ip: string, latitude: double, lcd: string, longitude: double, scale: string, temp: bigint, timestamp: bigint]


In [5]:
df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [6]:
df.createOrReplaceTempView("iot")

In [7]:
spark.sql("select cn, count(*) from iot group by cn").show()

+--------------------+--------+
|                  cn|count(1)|
+--------------------+--------+
|              Russia|    5989|
|            Paraguay|      32|
|            Anguilla|       7|
|               Macao|      33|
| U.S. Virgin Islands|      51|
|               Yemen|      19|
|British Indian Oc...|       2|
|             Senegal|      25|
|              Sweden|    2880|
|   Republic of Korea|   11879|
|              Guyana|       5|
|         Philippines|     681|
|              Jersey|      34|
|           Singapore|    1076|
|            Malaysia|     726|
|                Fiji|      11|
|              Turkey|     681|
|              Malawi|       9|
|               Åland|      20|
|                Iraq|       7|
+--------------------+--------+
only showing top 20 rows



In [8]:
spark.sql("select distinct cn from iot where c02_level > 1400 order by 1 desc").show()

+--------------------+
|                  cn|
+--------------------+
|               Åland|
|            Zimbabwe|
|              Zambia|
|               Yemen|
|             Vietnam|
|           Venezuela|
|             Vanuatu|
|          Uzbekistan|
|             Uruguay|
|       United States|
|      United Kingdom|
|United Arab Emirates|
|             Ukraine|
|              Uganda|
| U.S. Virgin Islands|
|        Turkmenistan|
|              Turkey|
|             Tunisia|
| Trinidad and Tobago|
|                Togo|
+--------------------+
only showing top 20 rows



In [11]:
spark.sql("select cca3, device_id, count(device_id) as cnt from iot where lcd='red' group by cca3, device_id order by cnt").show()

+----+---------+---+
|cca3|device_id|cnt|
+----+---------+---+
| CHN|     2080|  1|
| PAK|     3563|  1|
| IND|     3661|  1|
| USA|     3766|  1|
| USA|     3881|  1|
| AUS|     4434|  1|
| USA|     4766|  1|
| USA|     4965|  1|
| BRA|     5595|  1|
| RUS|     7755|  1|
| CYP|     9042|  1|
| LUX|     9657|  1|
| ARG|     9985|  1|
| AUT|    10278|  1|
| FIN|    14431|  1|
| USA|    15631|  1|
| POL|    21087|  1|
| USA|    21733|  1|
| USA|    22433|  1|
| USA|    25130|  1|
+----+---------+---+
only showing top 20 rows



In [12]:
spark.sql("select device_id, cn from iot where battery_level=0").show()

+---------+-----------------+
|device_id|               cn|
+---------+-----------------+
|        8|            Japan|
|       12|    United States|
|       17|    United States|
|       44|          Germany|
|       80|           Canada|
|       92|Republic of Korea|
|       98|    United States|
|      107|            Japan|
|      111|        Australia|
|      116|    United States|
|      117|            China|
|      123|    United States|
|      131|    United States|
|      134|    United States|
|      140|          Germany|
|      151|            China|
|      156|            China|
|      167|    United States|
|      179|    United States|
|      180|     South Africa|
+---------+-----------------+
only showing top 20 rows

